### Model Creation and Creation of Visualization Methods

In [2]:
from ultralytics import YOLO

import cv2
import matplotlib.pyplot as plt


In [2]:
model = YOLO("yolo11n.pt")


In [2]:
def visualize_results(results): 
    for result in results:
        img = result.orig_img
    
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = result.names[int(box.cls[0])]
            confidence = box.conf[0]
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box
            cv2.putText(img, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

In [4]:
predictions = model.predict(source="./datasets/DAWN/Fog/foggy-011.jpg", show=True)

2024-11-22 17:30:31.792 Python[60288:14105010] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-22 17:30:31.792 Python[60288:14105010] +[IMKInputSession subclass]: chose IMKInputSession_Modern


image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/foggy-011.jpg: 384x640 1 car, 7 traffic lights, 39.9ms
Speed: 1.6ms preprocess, 39.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


In [5]:
import os
import numpy as np


def load_ground_truth(file_path, img_width, img_height):
    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)
            x2 = int((x_center + width / 2) * img_width)
            y2 = int((y_center + height / 2) * img_height)
            boxes.append((int(class_id), x1, y1, x2, y2))
    return boxes

def calculate_iou(boxA, boxB):
    xA = max(boxA[1], boxB[1])
    yA = max(boxA[2], boxB[2])
    xB = min(boxA[3], boxB[3])
    yB = min(boxA[4], boxB[4])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    boxAArea = (boxA[3] - boxA[1]) * (boxA[4] - boxA[2])
    boxBArea = (boxB[3] - boxB[1]) * (boxB[4] - boxB[2])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def evaluate_model_on_folder(folder_path, gt_folder, model):
    all_ious = []
    for img_name in os.listdir(folder_path):
        if not img_name.endswith(".jpg"):
            continue
        img_path = os.path.join(folder_path, img_name)
        gt_path = os.path.join(gt_folder, os.path.splitext(img_name)[0] + ".txt")
        
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[:2]
        
        gt_boxes = load_ground_truth(gt_path, img_width, img_height)
        results = model.predict(source=img_path, show=False)
        
        for pred in results[0].boxes:
            pred_class = int(pred.cls[0])
            pred_box = [pred_class] + list(map(int, pred.xyxy[0].tolist()))
            best_iou = 0.0
            for gt_box in gt_boxes:
                if pred_class == gt_box[0]:
                    iou = calculate_iou(pred_box, gt_box)
                    best_iou = max(best_iou, iou)
            all_ious.append(best_iou)
    
    mean_iou = np.mean(all_ious)
    return mean_iou



### Finding the class mappings

In [15]:
class_names = model.names  # Dictionary {index: 'classname'}
print("Class Indices and Names:")
for idx, name in class_names.items():
    print(f"Index: {idx}, Name: {name}")

Class Indices and Names:
Index: 0, Name: person
Index: 1, Name: bicycle
Index: 2, Name: car
Index: 3, Name: motorcycle
Index: 4, Name: airplane
Index: 5, Name: bus
Index: 6, Name: train
Index: 7, Name: truck
Index: 8, Name: boat
Index: 9, Name: traffic light
Index: 10, Name: fire hydrant
Index: 11, Name: stop sign
Index: 12, Name: parking meter
Index: 13, Name: bench
Index: 14, Name: bird
Index: 15, Name: cat
Index: 16, Name: dog
Index: 17, Name: horse
Index: 18, Name: sheep
Index: 19, Name: cow
Index: 20, Name: elephant
Index: 21, Name: bear
Index: 22, Name: zebra
Index: 23, Name: giraffe
Index: 24, Name: backpack
Index: 25, Name: umbrella
Index: 26, Name: handbag
Index: 27, Name: tie
Index: 28, Name: suitcase
Index: 29, Name: frisbee
Index: 30, Name: skis
Index: 31, Name: snowboard
Index: 32, Name: sports ball
Index: 33, Name: kite
Index: 34, Name: baseball bat
Index: 35, Name: baseball glove
Index: 36, Name: skateboard
Index: 37, Name: surfboard
Index: 38, Name: tennis racket
Index:

In [9]:
import shutil
import random

In [9]:
model = YOLO("yolo11n.pt")

# Evaluate before training

In [31]:
train_dir = "./datasets/DAWN_split/train"
val_dir = "./datasets/DAWN_split/val"

### Evaluating the undtrained model on DAWN val dataset

In [35]:
print("Evaluating before training...")
initial_iou = evaluate_model_on_folder("./datasets/Dawn.v1i.yolov11/valid/images", "./datasets/Dawn.v1i.yolov11/valid/labels", model)
print("Initial Mean IoU on validation set:", initial_iou)

Evaluating before training...

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/haze-013_jpg.rf.0e7fec18de946008e2e0ceee60cc45e7.jpg: 640x640 (no detections), 113.0ms
Speed: 30.2ms preprocess, 113.0ms inference, 6.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/mist-098_jpg.rf.9f0bab09b56e4b13d6cf6e6c72cd88a2.jpg: 640x640 1 car, 61.6ms
Speed: 2.1ms preprocess, 61.6ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/sand_storm-281_jpg.rf.434f3007f121b710dde7af89981161e3.jpg: 640x640 6 cars, 1 truck, 58.3ms
Speed: 1.5ms preprocess, 58.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/rain_storm-265

In [36]:
for k, v in model.named_parameters():
    print(k)

model.model.0.conv.weight
model.model.0.conv.bias
model.model.1.conv.weight
model.model.1.conv.bias
model.model.2.cv1.conv.weight
model.model.2.cv1.conv.bias
model.model.2.cv2.conv.weight
model.model.2.cv2.conv.bias
model.model.2.m.0.cv1.conv.weight
model.model.2.m.0.cv1.conv.bias
model.model.2.m.0.cv2.conv.weight
model.model.2.m.0.cv2.conv.bias
model.model.3.conv.weight
model.model.3.conv.bias
model.model.4.cv1.conv.weight
model.model.4.cv1.conv.bias
model.model.4.cv2.conv.weight
model.model.4.cv2.conv.bias
model.model.4.m.0.cv1.conv.weight
model.model.4.m.0.cv1.conv.bias
model.model.4.m.0.cv2.conv.weight
model.model.4.m.0.cv2.conv.bias
model.model.5.conv.weight
model.model.5.conv.bias
model.model.6.cv1.conv.weight
model.model.6.cv1.conv.bias
model.model.6.cv2.conv.weight
model.model.6.cv2.conv.bias
model.model.6.m.0.cv1.conv.weight
model.model.6.m.0.cv1.conv.bias
model.model.6.m.0.cv2.conv.weight
model.model.6.m.0.cv2.conv.bias
model.model.6.m.0.cv3.conv.weight
model.model.6.m.0.cv3.

### Training the model on DAWN train dataset

In [ ]:
# Train the model on the split data
model = YOLO("yolo11n.pt")
print("Training model...")
results = model.train(data="/Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/data.yaml", epochs=50, imgsz=400, device="mps", pretrained=True, lr0=1e-4, plots=True)  # Adjust epochs as needed


I ran out of memory here(

In [ ]:
import torch


In [51]:
torch.save(model.state_dict(), 'yolov11x_model_state.pt')

In [15]:
model = YOLO("/Users/vahe/Downloads/DL_homeworks/semester_project/runs/detect/train13/weights/best.pt")

### Evaluating the find tuned model

In [16]:
# Evaluate after training
print("Evaluating after training...")
final_iou = evaluate_model_on_folder("./datasets/Dawn.v1i.yolov11/valid/images", "./datasets/Dawn.v1i.yolov11/valid/labels", model)
print("Final Mean IoU on validation set:", final_iou)

Evaluating after training...

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/haze-013_jpg.rf.0e7fec18de946008e2e0ceee60cc45e7.jpg: 416x416 1 0, 3 2s, 47.0ms
Speed: 5.7ms preprocess, 47.0ms inference, 4.7ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/mist-098_jpg.rf.9f0bab09b56e4b13d6cf6e6c72cd88a2.jpg: 416x416 2 2s, 35.1ms
Speed: 1.1ms preprocess, 35.1ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/sand_storm-281_jpg.rf.434f3007f121b710dde7af89981161e3.jpg: 416x416 6 2s, 34.5ms
Speed: 0.8ms preprocess, 34.5ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/rain_storm-265_jpg.rf.5fde1cbfde0731

In [5]:
model = YOLO("/Users/vahe/Downloads/DL_homeworks/semester_project/runs/detect/train13/weights/best.pt")


### Training the model on synthetic dataset

In [ ]:
results = model.train(data="/Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/data.yaml", epochs=50, imgsz=400, device="mps", pretrained=True, lr0=5e-4,   plots=True)  # Adjust epochs as needed


New https://pypi.org/project/ultralytics/8.3.48 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.28 🚀 Python-3.12.4 torch-2.5.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=/Users/vahe/Downloads/DL_homeworks/semester_project/runs/detect/train13/weights/best.pt, data=/Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=400, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buff

train: Scanning /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/train/labels... 5805 images, 93 backgrounds, 0 corrupt: 100%|██████████| 5805/5805 [00:01<00:00, 3637.08it/s]


train: New cache created: /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/train/labels.cache


val: Scanning /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/valid/labels... 549 images, 8 backgrounds, 0 corrupt: 100%|██████████| 549/549 [00:00<00:00, 3219.66it/s]

val: New cache created: /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/valid/labels.cache


Plotting labels to runs/detect/train14/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to runs/detect/train14
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.65G      2.037      2.537      1.374        335        416:   5%|▌         | 19/363 [00:27<14:20,  2.50s/it]

In [4]:
model = YOLO("/Users/vahe/Downloads/DL_homeworks/semester_project/runs/detect/train14/weights/best.pt")


In [4]:
print("Evaluating after training on synthetic dataset ...")
final_iou = evaluate_model_on_folder("./datasets/Dawn.v1i.yolov11/valid/images", "./datasets/Dawn.v1i.yolov11/valid/labels", model)
print("Final Mean IoU on validation set:", final_iou)

Evaluating after training on synthetic dataset ...

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/haze-013_jpg.rf.0e7fec18de946008e2e0ceee60cc45e7.jpg: 416x416 1 person, 56.5ms
Speed: 3.3ms preprocess, 56.5ms inference, 6.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/mist-098_jpg.rf.9f0bab09b56e4b13d6cf6e6c72cd88a2.jpg: 416x416 (no detections), 33.0ms
Speed: 0.9ms preprocess, 33.0ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/sand_storm-281_jpg.rf.434f3007f121b710dde7af89981161e3.jpg: 416x416 1 bus, 3 cars, 43.4ms
Speed: 2.4ms preprocess, 43.4ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/im

In [ ]:
results = model.train(data="/Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/data.yaml", epochs=10, imgsz=400, device="mps", pretrained=True, lr0=5e-4,   plots=True)  # Adjust epochs as needed


New https://pypi.org/project/ultralytics/8.3.48 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.28 🚀 Python-3.12.4 torch-2.5.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=/Users/vahe/Downloads/DL_homeworks/semester_project/runs/detect/train14/weights/best.pt, data=/Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=400, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buff

train: Scanning /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/train/labels.cache... 5805 images, 93 backgrounds, 0 corrupt: 100%|██████████| 5805/5805 [00:00<?, ?it/s]
val: Scanning /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/valid/labels... 549 images, 8 backgrounds, 0 corrupt: 100%|██████████| 549/549 [00:00<00:00, 3579.93it/s]

val: New cache created: /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/traffic_detection_project/valid/labels.cache


Plotting labels to runs/detect/train15/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to runs/detect/train15
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      2.81G      1.393      1.133      1.078        208        416:  23%|██▎       | 83/363 [01:23<04:25,  1.05it/s]

### Evaluation and comparison of fine tuned models on test dataset and DAWN dataset

In [23]:
model = YOLO("/Users/vahe/Downloads/DL_homeworks/semester_project/runs/detect/train14/weights/best.pt")


In [12]:
print("Evaluating after training on synthetic dataset 10 iterations more...")
final_iou = evaluate_model_on_folder("./datasets/Dawn.v1i.yolov11/valid/images", "./datasets/Dawn.v1i.yolov11/valid/labels", model)
print("Final Mean IoU on validation set:", final_iou)

Evaluating after training on synthetic dataset 10 iterations more...

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/haze-013_jpg.rf.0e7fec18de946008e2e0ceee60cc45e7.jpg: 416x416 1 person, 50.5ms
Speed: 1.2ms preprocess, 50.5ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/mist-098_jpg.rf.9f0bab09b56e4b13d6cf6e6c72cd88a2.jpg: 416x416 (no detections), 38.8ms
Speed: 0.9ms preprocess, 38.8ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/sand_storm-281_jpg.rf.434f3007f121b710dde7af89981161e3.jpg: 416x416 1 bus, 3 cars, 42.6ms
Speed: 1.0ms preprocess, 42.6ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1

In [13]:
print("Evaluating after training on synthetic dataset 10 iterations more on dawn train set...")
final_iou = evaluate_model_on_folder("./datasets/Dawn.v1i.yolov11/train/images", "./datasets/Dawn.v1i.yolov11/train/labels", model)
print("Final Mean IoU on dawn training set:", final_iou)

Evaluating after training on synthetic dataset 10 iterations more on dawn train set...

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/train/images/haze-081_jpg.rf.5482666a3ecb0e7319fb000e047a711a.jpg: 416x416 4 cars, 1 person, 56.6ms
Speed: 4.0ms preprocess, 56.6ms inference, 3.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/train/images/sand_storm-103_jpg.rf.077b8f587865968d48217ba2dcd08181.jpg: 416x416 3 cars, 38.0ms
Speed: 0.8ms preprocess, 38.0ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/train/images/mist-158_jpg.rf.a36b47beb454ab22d4b04d567804e737.jpg: 416x416 13 cars, 34.3ms
Speed: 0.9ms preprocess, 34.3ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datas

In [14]:
orig_model = YOLO("yolo11n.pt")

In [28]:
print("Original model on carr.v1i dataset")
final_iou = evaluate_model_on_folder("./datasets/carr.v1i.yolov11/valid/images", "./datasets/carr.v1i.yolov11/valid/labels", orig_model)
print("Final Mean IoU on dawn validation set:", final_iou)

Original model on carr.v1i dataset

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/291_jpeg.rf.bab2e78c3f7320e7dcae9371066f002e.jpg: 480x640 (no detections), 55.2ms
Speed: 4.9ms preprocess, 55.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/417_png.rf.92ae11f6f8ae59428ce23844cdec91f2.jpg: 480x640 (no detections), 53.9ms
Speed: 1.0ms preprocess, 53.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/454_jpeg.rf.5f220be5b6ca776c922d3f0be898af2a.jpg: 384x640 1 car, 42.5ms
Speed: 1.1ms preprocess, 42.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/254_jpeg.rf.8dead10fe53b51dfd54

In [29]:
print("Original model on carr.v1i dataset")
final_iou = evaluate_model_on_folder("./datasets/carr.v1i.yolov11/valid/images", "./datasets/carr.v1i.yolov11/valid/labels", model)
print("Final Mean IoU on dawn validation set:", final_iou)

Original model on carr.v1i dataset

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/291_jpeg.rf.bab2e78c3f7320e7dcae9371066f002e.jpg: 320x416 (no detections), 43.1ms
Speed: 1.1ms preprocess, 43.1ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/417_png.rf.92ae11f6f8ae59428ce23844cdec91f2.jpg: 320x416 (no detections), 38.7ms
Speed: 0.9ms preprocess, 38.7ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/454_jpeg.rf.5f220be5b6ca776c922d3f0be898af2a.jpg: 256x416 (no detections), 29.9ms
Speed: 1.4ms preprocess, 29.9ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/valid/images/254_jpeg.rf.8dead10fe

In [25]:
print("Original model on carr.v1i dataset")
final_iou = evaluate_model_on_folder("./datasets/carr.v1i.yolov11/train/images", "./datasets/carr.v1i.yolov11/train/labels", orig_model)
print("Final Mean IoU on dawn training set:", final_iou)

Original model on carr.v1i dataset

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/1844_jpeg.rf.a2c5f947c3ebe1040e65b02bd6cf88d4.jpg: 384x640 1 car, 66.6ms
Speed: 2.9ms preprocess, 66.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/2121_jpg.rf.c279926842d35837d195c5ceba301d31.jpg: 448x640 (no detections), 42.4ms
Speed: 1.5ms preprocess, 42.4ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/1614_jpeg.rf.a679b561914eacfca4d03fe56dd0425d.jpg: 384x640 1 car, 35.9ms
Speed: 1.7ms preprocess, 35.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/444_png.rf.632fa432477733a35a2be51768e

In [27]:
print("Original model on carr.v1i dataset")
final_iou = evaluate_model_on_folder("./datasets/carr.v1i.yolov11/train/images", "./datasets/carr.v1i.yolov11/train/labels", model)
print("Final Mean IoU on dawn training set:", final_iou)

Original model on carr.v1i dataset

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/1844_jpeg.rf.a2c5f947c3ebe1040e65b02bd6cf88d4.jpg: 256x416 1 car, 31.1ms
Speed: 1.0ms preprocess, 31.1ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/2121_jpg.rf.c279926842d35837d195c5ceba301d31.jpg: 288x416 (no detections), 33.7ms
Speed: 0.6ms preprocess, 33.7ms inference, 0.2ms postprocess per image at shape (1, 3, 288, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/1614_jpeg.rf.a679b561914eacfca4d03fe56dd0425d.jpg: 256x416 (no detections), 25.8ms
Speed: 0.8ms preprocess, 25.8ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/carr.v1i.yolov11/train/images/444_png.rf.632fa432477733a35